In [19]:
# ATSC 500 Project Code 
# Reading ERA5 grib files and saving min/max temp each day
# Mina Deshler

import numpy as np
import pandas as pd
import os 
# from glob import glob
import cfgrib
import xarray as xr
import pygrib
import matplotlib.pyplot as plt

In [4]:
### Where to get ERA5 meteorological reanalysis data: 
    ### https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-land?tab=form
    
### Reference material: https://jswhit.github.io/pygrib/api.html
    ### https://docs.xarray.dev/en/stable/examples/ERA5-GRIB-example.html 

#Test with one file 
folder = r"C:\Users\minad\Documents\UBC\ATSC-500\project\ERA5\adaptor.mars.internal-1719541798.5122192-13537-12-1eebd5af-d2a5-4d2f-8508-fe7a5c0eea6d.grib"

#f = os.path.join(directory, file)
FILE_NAME = os.fsdecode(folder)
ds = cfgrib.open_datasets(FILE_NAME, engine="cfgrib")

ds

[<xarray.Dataset>
 Dimensions:     (time: 32, step: 24, latitude: 113, longitude: 251)
 Coordinates:
     number      int32 0
   * time        (time) datetime64[ns] 2014-07-31 2014-08-01 ... 2014-08-31
   * step        (step) timedelta64[ns] 01:00:00 02:00:00 ... 1 days 00:00:00
     surface     float64 0.0
   * latitude    (latitude) float64 60.0 59.9 59.8 59.7 ... 49.1 49.0 48.9 48.8
   * longitude   (longitude) float64 -139.0 -138.9 -138.8 ... -114.1 -114.0
     valid_time  (time, step) datetime64[ns] 2014-07-31T01:00:00 ... 2014-09-01
 Data variables:
     sshf        (time, step, latitude, longitude) float32 ...
     t2m         (time, step, latitude, longitude) float32 ...
     d2m         (time, step, latitude, longitude) float32 ...
 Attributes:
     GRIB_edition:            1
     GRIB_centre:             ecmf
     GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
     GRIB_subCentre:          0
     Conventions:             CF-1.7
     institution:  

In [5]:
# Loop through and extract data from ERA5 downloaded data; save as one large xarray
folder = r"C:\Users\minad\Documents\UBC\ATSC-500\project\ERA5"
ext = '.grib'
datasets = []

for file in os.listdir(folder):
    #Iterate through ERA5 grib files 
    if file.endswith(ext):
        f = os.path.join(folder,file)
        #print(f)
        data = xr.load_dataset(f, engine="cfgrib")
        datasets.append(data)
    else:
        continue #to ignore the .idx files that are saved

In [16]:
# Ref: https://stackoverflow.com/questions/33435953/is-it-possible-to-append-to-an-xarray-dataset 
# Ref: https://stackoverflow.com/questions/21317384/how-to-concatenate-two-dataframes-without-duplicates 

#datasave = xr.concat(datasets,dim="time")
#datasave = xr.concat(datasets,dim="time").max(dim='time') #there are some repeats for the data 
datasave = xr.concat(datasets,dim="time").drop_duplicates(dim="time") #.reset_index(dims_or_levels="time",drop=True)
datasave

<xarray.Dataset>
Dimensions:     (time: 994, step: 24, latitude: 113, longitude: 251)
Coordinates:
    number      int32 0
  * time        (time) datetime64[ns] 1998-05-31 1998-06-01 ... 2014-08-31
  * step        (step) timedelta64[ns] 01:00:00 02:00:00 ... 1 days 00:00:00
    surface     float64 0.0
  * latitude    (latitude) float64 60.0 59.9 59.8 59.7 ... 49.1 49.0 48.9 48.8
  * longitude   (longitude) float64 -139.0 -138.9 -138.8 ... -114.1 -114.0
    valid_time  (time, step) datetime64[ns] 1998-05-31T01:00:00 ... 2014-09-01
Data variables:
    d2m         (time, step, latitude, longitude) float32 nan nan ... nan nan
    t2m         (time, step, latitude, longitude) float32 nan nan ... nan nan
    sshf        (time, step, latitude, longitude) float32 nan nan ... nan nan
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-07-27T14:33 GRIB to CDM+CF via cfgrib-0.9.1...

In [17]:
df = datasave.to_dataframe()

In [18]:
# Save both xarray format and pandas dataframe to use later:
df.head()
#zarr
datasave.load().to_zarr("era5_2.zarr")
#pandas
#path = r"C:\Users\minad\Documents\UBC\ATSC-500\project\ERA5\ERA5.pkl"
#df.to_pickle(path)

In [2]:
# Load in all era5 data
era5 = xr.open_zarr('era5.zarr')

In [8]:
#Save only max and min values per day
Tmin = era5.t2m.min(dim = "step")
Tmax = era5.t2m.max(dim = "step")
SSHFmin = era5.sshf.min(dim = "step")
SSHFmax = era5.sshf.max(dim = "step")

In [15]:
Tmin

NameError: name 'Tmin' is not defined

In [17]:
# Ref: https://docs.xarray.dev/en/latest/user-guide/indexing.html 
Tmin[0,0,0]

<xarray.DataArray 't2m' ()>
dask.array<getitem, shape=(), dtype=float32, chunksize=(), chunktype=numpy.ndarray>
Coordinates:
    latitude   float64 60.0
    longitude  float64 -139.0
    number     int32 ...
    surface    float64 ...
    time       datetime64[ns] 1998-05-31

In [15]:
Tmax.isel(time=[0])

<xarray.DataArray 't2m' (time: 1, latitude: 113, longitude: 251)>
dask.array<getitem, shape=(1, 113, 251), dtype=float32, chunksize=(1, 29, 63), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float64 60.0 59.9 59.8 59.7 ... 49.1 49.0 48.9 48.8
  * longitude  (longitude) float64 -139.0 -138.9 -138.8 ... -114.2 -114.1 -114.0
    number     int32 ...
    surface    float64 ...
  * time       (time) datetime64[ns] 1998-05-31

In [19]:
cmiptemps = xr.open_zarr('maxmintemps.zarr')
cmiptemps

<xarray.Dataset>
Dimensions:         (dcpp_init_year: 1, lat: 11, bnds: 2, lon: 16,
                     member_id: 1, time: 0)
Coordinates:
  * dcpp_init_year  (dcpp_init_year) float64 nan
  * lat             (lat) float64 49.5 50.5 51.5 52.5 ... 56.5 57.5 58.5 59.5
    lat_bnds        (lat, bnds) float64 dask.array<chunksize=(11, 2), meta=np.ndarray>
  * lon             (lon) float64 225.6 226.9 228.1 229.4 ... 241.9 243.1 244.4
    lon_bnds        (lon, bnds) float64 dask.array<chunksize=(16, 2), meta=np.ndarray>
  * member_id       (member_id) object 'r1i1p1f1'
  * time            (time) int64 
    time_bnds       (time, bnds) float64 dask.array<chunksize=(0, 2), meta=np.ndarray>
Dimensions without coordinates: bnds
Data variables:
    tasmax          (member_id, dcpp_init_year, time, lat, lon) float32 dask.array<chunksize=(1, 1, 0, 11, 16), meta=np.ndarray>
    tasmin          (member_id, dcpp_init_year, time, lat, lon) float32 dask.array<chunksize=(1, 1, 0, 11, 16), meta=np.ndarray>
Attributes: (12/55)
    Conventions:                      CF-1.7 CMIP-6.0 UGRID-1.0
    activity_id:                      CMIP
    branch_method:                    standard
    branch_time_in_child:             0.0
    branch_time_in_parent:            54750.0
    comment:                          <null ref>
    ...                               ...
    sub_experiment_id:                none
    table_id:                         day
    title:                            NOAA GFDL GFDL-CM4 model output prepare...
    variant_info:                     N/A
    variant_label:                    r1i1p1f1
    version_id:                       v20180701